<a href="https://colab.research.google.com/github/sadiashoaib415/Quantum_Computer/blob/main/Monte_Carlo_Qiskit_Code01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1️⃣ Install necessary libraries (run once in Colab)
!pip install qiskit qiskit-aer qiskit-algorithms qiskit-finance -q

# 2️⃣ Imports
import numpy as np
from scipy.stats import norm
from qiskit.primitives import StatevectorSampler
from qiskit_algorithms import IterativeAmplitudeEstimation
from qiskit_finance.circuit.library import LogNormalDistribution
from qiskit_finance.applications.estimation import EuropeanCallPricing


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.4/647.4 kB 12.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.8/327.8 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.1/237.1 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 3.7 MB/s eta 0:00:00


In [2]:
# --- Parameters ---
num_uncertainty_qubits = 3  # Resolution of price grid (2^3 = 8 points)
spot_price = 2.0            # Current asset price
strike_price = 2.0          # Option exercise price
volatility = 0.30           # 30% volatility
risk_free_rate = 0.05       # 5% risk-free interest rate
maturity = 40 / 365         # 40 days → in years

# --- Log-normal distribution parameters ---
mean = (risk_free_rate - 0.5 * volatility**2) * maturity + np.log(spot_price)
stddev = np.sqrt(volatility**2 * maturity)

# Price bounds: ±3σ covers ~99% of possible prices
low = np.exp(mean - 3 * stddev)
high = np.exp(mean + 3 * stddev)
bounds = (low, high)

# --- Uncertainty model ---
uncertainty_model = LogNormalDistribution(
    num_uncertainty_qubits,
    mu=mean,
    sigma=stddev**2,  # Qiskit now expects variance
    bounds=bounds
)


In [3]:
# --- European Call Pricing ---
european_call = EuropeanCallPricing(
    num_state_qubits=num_uncertainty_qubits,
    strike_price=strike_price,
    rescaling_factor=bounds[1] - strike_price,
    uncertainty_model=uncertainty_model,
    bounds=bounds
)

# Convert to estimation problem (new API)
problem = european_call.to_estimation_problem()


In [4]:
# --- Amplitude Estimation ---
sampler = StatevectorSampler()

iae = IterativeAmplitudeEstimation(
    epsilon_target=0.01,  # 1% target precision
    alpha=0.05,           # 95% confidence level
    sampler=sampler
)

# Run quantum estimation
result = iae.estimate(problem)


In [5]:
# Quantum price
quantum_price = european_call.interpret(result)

# Classical Black-Scholes formula
d1 = (np.log(spot_price / strike_price) + (risk_free_rate + 0.5 * volatility**2) * maturity) / (volatility * np.sqrt(maturity))
d2 = d1 - volatility * np.sqrt(maturity)
bs_price = (spot_price * norm.cdf(d1) - strike_price * np.exp(-risk_free_rate * maturity) * norm.cdf(d2))

# Results
print(f"--- Quantum vs Classical ---")
print(f"Quantum Estimated Price: ${quantum_price:.4f}")
print(f"Classical BS Price:      ${bs_price:.4f}")
print(f"Difference:              ${abs(quantum_price - bs_price):.4f}")
print(f"Confidence Interval:     {[round(x,4) for x in result.confidence_interval_processed]}")


--- Quantum vs Classical ---
Quantum Estimated Price: $0.1239
Classical BS Price:      $0.0846
Difference:              $0.0394
Confidence Interval:     [np.float64(0.1224), np.float64(0.1254)]
